<a href="https://colab.research.google.com/github/ethandavenport/Optimization-I-Project-2/blob/main/mip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 55.7 MB/s eta 0:00:00


In [2]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

In [3]:
url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/roi_company1.csv"
df_roi1 = pd.read_csv(url)

url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/roi_company2.csv"
df_roi2 = pd.read_csv(url)

url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/roi_monthly.csv"
df_roi_monthly = pd.read_csv(url)

url = "https://raw.githubusercontent.com/ethandavenport/Optimization-I-Project-2/main/min_amount.csv"
df_min = pd.read_csv(url)